<h1>Title</h1>

<h1>Introduction</h1>

<p>Placeholder text</p>

<h1>Methods and Results</h1>

<p>Describe Methods here</p>
<p><b>MAKE SURE TO LABEL VISUALS (ex. Figure 1) and refer back to them in explanations</b></p>

In [ ]:
# Load the data set: 


<p>PlaceHolder</p>

In [ ]:
# Split the data into training and testing data 


<p>PlaceHolder</p>

In [ ]:
#Code

<p>PlaceHOlder
</p>

In [ ]:
#Code

<p>PlaceHolder</p>

<h1>Discussion</h1>

<p>PlaceHolder
</p>

<h1>References</h1>

<p>APA format??????</p>